# Génération des corrections d'un modèle BERT pré-entraîné
Le jeu de données utilisé est CoNLL-2014.  
La majorité du code est basé sur le répertoire github (https://github.com/kanekomasahiro/bert-gec) de l'article Encoder-Decoder Models Can Benefit from Pre-trained Masked Language Models in Grammatical Error Correction [Kaneko
et al., 2020].

# Installations et importations

In [ ]:
!wget https://raw.githubusercontent.com/kanekomasahiro/bert-gec/master/scripts/convert_m2_to_parallel.py
!wget https://raw.githubusercontent.com/kanekomasahiro/bert-gec/master/scripts/detok.py
!wget https://raw.githubusercontent.com/kanekomasahiro/bert-gec/master/scripts/generate.sh
!wget https://raw.githubusercontent.com/kanekomasahiro/bert-gec/master/scripts/setup.sh
!wget https://raw.githubusercontent.com/kanekomasahiro/bert-gec/master/scripts/train.sh

In [ ]:
!pip install boto3

In [ ]:
import shutil
import os

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
os.mkdir("/bert-fuse/")

shutil.copy("/content/drive/MyDrive/official-2014.combined.m2", "/content/")
shutil.copy("/content/drive/MyDrive/dict.src.txt", "/bert-fuse/")
shutil.copy("/content/drive/MyDrive/dict.trg.txt", "/bert-fuse/")
shutil.copy("/content/drive/MyDrive/checkpoint_best.pt", "/bert-fuse/")

# Génération des fichiers sources et cibles

In [ ]:
!python convert_m2_to_parallel.py official-2014.combined.m2 output_src.txt output_tgt.txt

In [ ]:
files.download('/content/output_src.txt')
files.download('/content/output_tgt.txt')

# BERT

## Préparation

In [ ]:
!sh setup.sh

Certains des fichiers téléchargés utilisent des versions antérieures de numpy et de pytorch. Ils sont modifiés ici pour être comptatibles avec les versions à jour.

In [ ]:
# Remplacement de np.float par np.float64
with open("../bert-nmt/fairseq/data/indexed_dataset.py", "r+") as f:
  content = f.read()
  modified_content = content.replace("np.float", "np.float64")
  f.seek(0)
  f.write(modified_content)


In [ ]:
# Les opérations entre les float et les bool ne sont plus supportées par pytorch
with open("../bert-nmt/fairseq/sequence_generator.py", "r+") as f:
  content = f.read()
  modified_content = content.replace("bert_outs, _ = model.models[0].bert_encoder(bertinput, output_all_encoded_layers=True, attention_mask= 1. - bert_encoder_padding_mask)",
                                     "bert_outs, _ = model.models[0].bert_encoder(bertinput, output_all_encoded_layers=True, attention_mask=~bert_encoder_padding_mask)")
  f.seek(0)
  f.truncate()
  f.write(modified_content)

In [ ]:
# torch.div ne supporte plus l'assignation d'un float à un double
with open("../bert-nmt/fairseq/search.py", "r+") as f:
  content = f.read()
  modified_content = content.replace("torch.div(self.indices_buf, vocab_size, out=self.beams_buf)",
                                     "self.beams_buf = self.indices_buf // vocab_size")
  f.seek(0)
  f.truncate()
  f.write(modified_content)


## Génération des prédictions du modèle BERT

In [ ]:
!sh generate.sh output_src.txt 0

In [ ]:
files.download('../bert-fuse/output/test.bert.src')
files.download('../bert-fuse/output/test.best.tok')
files.download('../bert-fuse/output/test.bpe.src')
files.download('../bert-fuse/output/test.cat.src')
files.download('../bert-fuse/output/test.nbest.tok')